In [1]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
from sentence_transformers import LoggingHandler
import logging

# Hyperparameters
limit = 4000

# GPU compatibility
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Fine-tuned model save path
fine_tuned_model_path = 'fine_tuned_sbert_model'

# Step 1: Load the STS-B dataset
print("Loading the STS-B dataset...")
dataset = load_dataset('glue', 'stsb')

# Limit the dataset size for faster training (batched processing)
def limit_rows(batch):
    return {
        'sentence1': batch['sentence1'][:limit],
        'sentence2': batch['sentence2'][:limit],
        'label': batch['label'][:limit]
    }

# Apply the row limit to train and validation datasets
print(f"Limiting dataset to {limit} rows...")
dataset['train'] = dataset['train'].select(range(min(limit, len(dataset['train']))))
dataset['validation'] = dataset['validation'].select(range(min(limit, len(dataset['validation']))))

# Step 2: Prepare the training data
train_sentences1 = dataset['train']['sentence1']
train_sentences2 = dataset['train']['sentence2']
train_scores = dataset['train']['label']

# Normalize scores to the 0-1 range
print("Normalizing scores...")
train_scores = [score / 5.0 for score in train_scores]

# Prepare validation data
val_sentences1 = dataset['validation']['sentence1']
val_sentences2 = dataset['validation']['sentence2']
val_scores = dataset['validation']['label']
val_scores = [score / 5.0 for score in val_scores]

# Step 3: Load the larger pre-trained SBERT model
print("Loading the original SBERT model...")
original_model = SentenceTransformer('stsb-bert-large').to(device)

# Step 4: Evaluate the original model
print("Evaluating the original SBERT model...")
original_evaluator = EmbeddingSimilarityEvaluator(val_sentences1, val_sentences2, val_scores)
evaluation_results = original_evaluator(original_model)

# Print results
def print_results(evaluation_results, model_name="SBERT model"):
    if isinstance(evaluation_results, dict):
        for metric, value in evaluation_results.items():
            print(f"{model_name} {metric}: {value:.4f}")
    else:
        print(f"{model_name} evaluation score: {evaluation_results:.4f}")

print_results(evaluation_results)

# Step 5: Fine-tune the model
# Create InputExamples
train_examples = [
    InputExample(texts=[s1, s2], label=score)
    for s1, s2, score in zip(train_sentences1, train_sentences2, train_scores)
]

# Create a DataLoader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

# Define the loss function
train_loss = losses.CosineSimilarityLoss(original_model)

# Set up logging
logging.basicConfig(
    format='%(asctime)s - %(message)s',
    level=logging.INFO,
    handlers=[LoggingHandler()]
)

# Fine-tune the model with progress bar and logging
print("Fine-tuning the SBERT model...")
original_model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=10,
    warmup_steps=100,
    show_progress_bar=True,
    output_path=fine_tuned_model_path
)
print(f"Fine-tuned model was saved at {fine_tuned_model_path}")

# Step 6: Evaluate the fine-tuned model
print("Evaluating the fine-tuned SBERT model...")
fine_tuned_model = SentenceTransformer(fine_tuned_model_path)
fine_tuned_evaluator = EmbeddingSimilarityEvaluator(val_sentences1, val_sentences2, val_scores)
fine_tuned_score = fine_tuned_evaluator(fine_tuned_model)
print_results(fine_tuned_score, "Fine-tuned SBERT model")

# Step 7: Compare results
print("\nEvaluation Comparison:")
print_results(evaluation_results, "Original SBERT model")
print_results(fine_tuned_score, "Fine-tuned SBERT model")


C:\Users\Stefan\Documents\GitHub\UBB-Y3-AI-Research\models\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda
GPU: NVIDIA GeForce GTX 1080 Ti
Loading the STS-B dataset...
Limiting dataset to 4000 rows...
Normalizing scores...
Loading the original SBERT model...
Evaluating the original SBERT model...
SBERT model pearson_cosine: 0.8796
SBERT model spearman_cosine: 0.8816
Fine-tuning the SBERT model...


Step,Training Loss
500,0.006600
1000,0.004700
1500,0.002400
2000,0.001600
2500,0.001200


2025-01-14 23:54:17,975 - Save model to fine_tuned_sbert_model


Fine-tuned model was saved at fine_tuned_sbert_model
Evaluating the fine-tuned SBERT model...
2025-01-14 23:54:21,286 - Use pytorch device_name: cuda
2025-01-14 23:54:21,286 - Load pretrained SentenceTransformer: fine_tuned_sbert_model
2025-01-14 23:54:22,775 - EmbeddingSimilarityEvaluator: Evaluating the model on the  dataset:
2025-01-14 23:54:31,649 - Cosine-Similarity :	Pearson: 0.8801	Spearman: 0.8816
Fine-tuned SBERT model pearson_cosine: 0.8801
Fine-tuned SBERT model spearman_cosine: 0.8816

Evaluation Comparison:
Original SBERT model pearson_cosine: 0.8796
Original SBERT model spearman_cosine: 0.8816
Fine-tuned SBERT model pearson_cosine: 0.8801
Fine-tuned SBERT model spearman_cosine: 0.8816
